# Imports & setup

In [1]:
import os
os.chdir("D:\PulpitE\FPL_ML")

In [2]:
import pandas as pd
import numpy as np
import pickle
from src.match_names import name_fbref_to_fpl, neutralize_name
from config import *

In [3]:
info = ["Name_original", "GW", "Season", "Team", "Opponent", "Was_home", "Team_rating", "Opp_rating", "FPL_pos", "Price"]

In [4]:
len(FEATURES_OUTFIELD)

38

# Reading from files

In [5]:
df = pd.read_csv("data/upcoming/upcoming_fixtures.csv")

C:\users\user\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3369: DtypeWarning: Columns (2,3,6,8,36,43,44,45,59,84,86,147,208) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
df = df[~df[FEATURES_OUTFIELD].isnull().any(axis=1)] # for some reason 45 rows are missing some feature values

In [7]:
X_test = df[df["Season"] == CURRENT_SEASON][FEATURES_OUTFIELD]

In [8]:
df.columns

Index(['Unnamed: 0', 'Date', 'Day', 'Venue', 'Team', 'Opponent', 'Name',
       'Start', 'Pos', 'Min',
       ...
       'FPL_xGC_30', 'Opp_rating_30', 'Team_rating_30', 'Min_points_30',
       'Team_result_30', 'DEF', 'FWD', 'MID', 'Finished', 'Price'],
      dtype='object', length=274)

In [9]:
X_test.columns

Index(['Was_home', 'Rating_difference', 'Value', 'Transfers_balance',
       'Avg_FPL_points', 'Min_2', 'Gls_2', 'Ast_2', 'xG_2', 'xA_2',
       'Team_CS_2', 'Team_score_2', 'Opp_score_2', 'Team_result_2',
       'FPL_points_2', 'Min_4', 'Gls_4', 'Ast_4', 'xG_4', 'xA_4', 'Team_CS_4',
       'Team_score_4', 'Opp_score_4', 'Team_result_4', 'FPL_points_4',
       'Min_30', 'Gls_30', 'Ast_30', 'xG_30', 'xA_30', 'Team_CS_30',
       'Team_score_30', 'Opp_score_30', 'Team_result_30', 'FPL_points_30',
       'DEF', 'FWD', 'MID'],
      dtype='object')

In [10]:
# df[df["FPL_pos"] == "GK"]

In [11]:
def new_predict(X):
    XGB_COMPONENT = 0.45
    OVERALL_COMPONENT = 0.5
    FORM_COMPONENT = 0.25
    
    model_pred = np.array(model.predict2(X))
    overall_pred = np.array([row["Avg_FPL_points"] for i, row in X.iterrows()])
    form_pred = np.array([row["FPL_points_4"] for i, row in X.iterrows()])
    
    return np.add( model_pred * XGB_COMPONENT, overall_pred * OVERALL_COMPONENT, form_pred * FORM_COMPONENT )

file = open("models/GBR.pkl",'rb')
model = pickle.load(file)

In [12]:
model

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=5,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.2, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=500, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=42, ...)

# Predicting

In [13]:
def adjust_for_injuries(df):
    active_players = pd.read_csv("data/misc/active_players.csv")
    injured_players = active_players[active_players["chance_of_playing_this_round"] == 0]["name"].to_list()
    df.loc[df["Name_original"].isin(injured_players),"Pred"] = 0
    return df

In [14]:
def adjust_goalkeepers(df):
    gks = pd.read_csv("data/misc/goalkeepers.csv")
    gk_names = gks["Name"].to_list()
    gk_names = [name_fbref_to_fpl(neutralize_name(n)) for n in gk_names]
    # print(gk_names)
    df.loc[(df["FPL_pos"] == "GK") & ~(df["Name_original"].apply(neutralize_name).isin(gk_names)), "Pred"] = 0
    return df

In [15]:
def get_predictions(model, df, X, all_remaining=False):
    # make predictions on the test data and glues them to the rest of the dataframe
    predictions = model.predict(X)
    df_predictions = df[(df["Season"] == CURRENT_SEASON)].reset_index(drop=True)
        
    df_predictions.loc[:, "Pred"] = predictions
    preds = df_predictions[info + ["Pred"]]
    
    preds = adjust_goalkeepers(preds)
    preds = adjust_for_injuries(preds).sort_values(by=["Pred"], ascending = False)
    
    return preds

In [16]:
preds = get_predictions(model, df, X_test)

C:\users\user\appdata\local\programs\python\python38\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [17]:
# preds[(preds["FPL_pos"] == "GK") & (preds["GW"] == NEXT_GAMEWEEK)].head(30)

In [18]:
# next gameweek
preds[preds["GW"] == NEXT_GAMEWEEK].head(20)

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,FPL_pos,Price,Pred
1223,André Onana,3.0,2023-24,Manchester Utd,Nott'ham Forest,1.0,1877.037842,1671.868774,GK,50.0,7.101460
4779,Erling Haaland,3.0,2023-24,Manchester City,Sheffield United,0.0,2080.850586,1637.473755,FWD,140.0,5.152132
11344,Mohamed Salah,3.0,2023-24,Liverpool,Newcastle Utd,0.0,1943.976074,1887.064819,MID,125.0,5.105978
5388,Gabriel Martinelli Silva,3.0,2023-24,Arsenal,Fulham,1.0,1921.462769,1745.365723,MID,80.0,5.103013
13640,Sandro Tonali,3.0,2023-24,Newcastle Utd,Liverpool,1.0,1887.064819,1943.976074,MID,55.0,4.974339
2371,Bukayo Saka,3.0,2023-24,Arsenal,Fulham,1.0,1921.462769,1745.365723,MID,86.0,4.909317
14895,Tomáš Souček,3.0,2023-24,West Ham,Brighton,0.0,1777.551270,1832.364258,MID,50.0,4.874752
10520,Martin Ødegaard,3.0,2023-24,Arsenal,Fulham,1.0,1921.462769,1745.365723,MID,85.0,4.513186
1187,Andrew Robertson,3.0,2023-24,Liverpool,Newcastle Utd,0.0,1943.976074,1887.064819,DEF,65.0,4.321380
8977,Kieran Trippier,3.0,2023-24,Newcastle Utd,Liverpool,1.0,1887.064819,1943.976074,DEF,65.0,4.207139


In [19]:
# all preds
preds.head(20)

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,FPL_pos,Price,Pred
1241,André Onana,21.0,2023-24,Manchester Utd,Tottenham,1.0,1877.037842,1817.656738,GK,50.0,7.10146
1249,André Onana,29.0,2023-24,Manchester Utd,Sheffield United,1.0,1877.037842,1637.473755,GK,50.0,7.10146
1236,André Onana,16.0,2023-24,Manchester Utd,Bournemouth,1.0,1877.037842,1662.536621,GK,50.0,7.10146
1223,André Onana,3.0,2023-24,Manchester Utd,Nott'ham Forest,1.0,1877.037842,1671.868774,GK,50.0,7.10146
1235,André Onana,15.0,2023-24,Manchester Utd,Chelsea,1.0,1877.037842,1791.018188,GK,50.0,7.10146
1243,André Onana,23.0,2023-24,Manchester Utd,West Ham,1.0,1877.037842,1777.551270,GK,50.0,7.10146
1232,André Onana,12.0,2023-24,Manchester Utd,Luton,1.0,1877.037842,1602.327637,GK,50.0,7.10146
1246,André Onana,26.0,2023-24,Manchester Utd,Fulham,1.0,1877.037842,1745.365723,GK,50.0,7.10146
1248,André Onana,28.0,2023-24,Manchester Utd,Everton,1.0,1877.037842,1699.505859,GK,50.0,7.10146
1228,André Onana,8.0,2023-24,Manchester Utd,Brentford,1.0,1877.037842,1822.547119,GK,50.0,7.10146


In [20]:
# points all season
pd.DataFrame(preds.groupby(["Name_original"])["Pred"].sum().sort_values(ascending=False)).head(30)

,Pred
Name_original,
André Onana,255.968157
Mohamed Salah,206.894929
Tomáš Souček,204.670847
Sandro Tonali,202.294696
Erling Haaland,199.247349
Gabriel Martinelli Silva,175.810749
Bukayo Saka,170.353148
Andrew Robertson,165.794354
Martin Ødegaard,161.026469


# Saving to file

In [21]:
preds.to_csv("predictions/preds_next_season.csv", index=False)

# Importance

In [22]:
# import shap
# explainer = shap.Explainer(model.predict, X_test[9000:10000])
# shap_values = explainer(X_test[9000:10000])

In [23]:
# shap.plots.bar(shap_values, max_display=15)

In [24]:
# # Salah vs Nottingham Forest
# shap.plots.waterfall(shap_values[502])

In [25]:
# shap.plots.waterfall(shap_values[470])